In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
import mysql.connector
import joblib
import numpy as np
import pandas as pd

# Load the trained model pipeline (with preprocessing included)
model = joblib.load('xgb_model.pkl')

app = Flask(__name__)
app.secret_key = "Shree"

def get_connection():
    return mysql.connector.connect(
        host='mydb-instance.c1qcga28q7o6.eu-north-1.rds.amazonaws.com',
        port=3306,
        user='Bhavesh',
        password='bhavesh1975',
        database='Dream_House_Valuation',
        connection_timeout=60
    )

def setup_database():
    connection = get_connection()
    cursor = connection.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS registers (
            Email VARCHAR(70) PRIMARY KEY,
            Full_Name CHAR(70),
            mobile VARCHAR(15),
            password VARCHAR(20)
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS house_data (
            Email VARCHAR(100),
            City VARCHAR(100),
            square_feet FLOAT,
            Bedroom INT,
            Property_type VARCHAR(40),
            Furnished VARCHAR(40),
            Year_Built INT,
            Floor_No INT,
            Age_of_Property INT,
            Nearby_Schools INT,
            Nearby_Hospitals INT,
            Public_Transport_Accessibility VARCHAR(10),
            Parking_Space BOOLEAN,
            Security BOOLEAN,
            Amenities VARCHAR(10),
            Facing VARCHAR(20),
            Owner_Type VARCHAR(40)
        )
    """)
    connection.commit()
    cursor.close()
    connection.close()

setup_database()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/interaction')
def interaction():
    if 'user_email' not in session:
        flash("Please log in first.")
        return redirect(url_for('home'))
    user_info = get_user_info(session['user_email'])
    return render_template('interaction2.html', user=user_info)

@app.route('/house_data', methods=['POST'])
def house_data():
    if 'user_email' not in session:
        flash("Please log in first.")
        return redirect(url_for('home'))

    user_email = session['user_email']

    try:
        form = request.form
        input_data = {
            'City': form.get('city', '').strip(),
            'Property_Type': form.get('property_type', '').strip(),
            'BHK': int(form.get('bedroom', 0)),
            'Furnished_Status': form.get('furnished', '').strip(),
            'Size_in_SqFt': float(form.get('square_feet', 0)),
            'Year_Built': int(form.get('year_built', 0)),
            'Floor_No': int(form.get('floor_no', 0)),
            'Age_of_Property': int(form.get('age_of_property', 0)),
            'Nearby_Schools': int(form.get('nearby_schools', 0)),
            'Nearby_Hospitals': int(form.get('nearby_hospitals', 0)),
            'Public_Transport_Accessibility': form.get('public_transport_accessibility', '').capitalize().strip(),
            'Parking_Space': 1 if form.get('parking_space', '').lower() == 'yes' else 0,
            'Security': 1 if form.get('security', '').lower() == 'yes' else 0,
            'Amenities': form.get('amenities', '').strip(),  # treat as string categorical
            'Facing': form.get('facing', '').strip(),
            'Owner_Type': form.get('owner_type', '').strip()
        }

        # Store in DB
        connection = get_connection()
        cursor = connection.cursor()
        cursor.execute('''
            INSERT INTO house_data (
                Email, City, square_feet, Bedroom, Property_type, Furnished,
                Year_Built, Floor_No, Age_of_Property, Nearby_Schools, Nearby_Hospitals,
                Public_Transport_Accessibility, Parking_Space, Security, Amenities,
                Facing, Owner_Type
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (
            user_email, input_data['City'], input_data['Size_in_SqFt'], input_data['BHK'],
            input_data['Property_Type'], input_data['Furnished_Status'],
            input_data['Year_Built'], input_data['Floor_No'], input_data['Age_of_Property'],
            input_data['Nearby_Schools'], input_data['Nearby_Hospitals'],
            input_data['Public_Transport_Accessibility'], input_data['Parking_Space'],
            input_data['Security'], input_data['Amenities'], input_data['Facing'],
            input_data['Owner_Type']
        ))
        connection.commit()

        # Prepare DataFrame for prediction
        input_df = pd.DataFrame([input_data])

        # Define categorical and numerical features exactly as model expects
        categorical_features = [
            'City', 'Property_Type', 'Furnished_Status',
            'Public_Transport_Accessibility', 'Amenities', 'Facing', 'Owner_Type'
        ]
        numerical_features = [
            'BHK', 'Size_in_SqFt', 'Year_Built', 'Floor_No',
            'Age_of_Property', 'Nearby_Schools', 'Nearby_Hospitals',
            'Parking_Space', 'Security'
        ]

        # Convert types correctly
        for col in categorical_features:
            input_df[col] = input_df[col].astype(str)
        for col in numerical_features:
            input_df[col] = pd.to_numeric(input_df[col], errors='coerce')

        # Check for missing numeric values after conversion
        for col in numerical_features:
            if input_df[col].isnull().any():
                raise ValueError(f"Invalid or missing numeric input for '{col}'.")

        # Predict using the loaded model pipeline
        log_price_pred = model.predict(input_df)[0]
        price_pred = np.expm1(log_price_pred)

        user_info = get_user_info(user_email)
        return render_template('interaction2.html', user=user_info, predicted_price=round(price_pred, 2))

    except Exception as e:
        print(f"🚨 Error: {e}")
        user_info = get_user_info(user_email)
        return render_template('interaction2.html', user=user_info, error=str(e))

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()




@app.route('/register', methods=['POST'])
def register():
    full_name = request.form['fullname']
    mobile = request.form['mobile']
    email = request.form['email']
    password = request.form['password']
    confirm_password = request.form['confirm_password']

    if password != confirm_password:
        flash("Passwords do not match.")
        return redirect(url_for('home'))

    try:
        connection = get_connection()
        cursor = connection.cursor()
        cursor.execute(
            "INSERT INTO registers (Full_Name, mobile, Email, password) VALUES (%s, %s, %s, %s)",
            (full_name, mobile, email, password)
        )
        connection.commit()
        flash("Registration successful.")
    except Exception as e:
        flash(f"Error: {e}")
    finally:
        cursor.close()
        connection.close()

    return redirect(url_for('home'))

@app.route('/login', methods=['POST'])
def login():
    email = request.form['email']
    password = request.form['password']

    try:
        connection = get_connection()
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM registers WHERE Email = %s AND password = %s", (email, password))
        result = cursor.fetchone()
    except Exception as e:
        flash(f"Error: {e}")
        return redirect(url_for('home'))
    finally:
        cursor.close()
        connection.close()

    if result:
        session['user_email'] = result[0]
        flash(f"Welcome {result[1]}")
        return redirect(url_for('interaction'))
    else:
        flash("Invalid email or password.")
        return redirect(url_for('home'))

@app.route('/logout')
def logout():
    session.pop('user_email', None)
    flash("Logged out successfully.")
    return redirect(url_for('home'))

@app.route('/history')
def history():
    if 'user_email' not in session:
        flash("Please log in first.")
        return redirect(url_for('home'))

    user_email = session['user_email']
    connection = get_connection()
    cursor = connection.cursor()

    cursor.execute('''
        SELECT City, square_feet, Bedroom, Property_type, Furnished,
               Year_Built, Floor_No, Age_of_Property, Nearby_Schools, Nearby_Hospitals,
               Public_Transport_Accessibility, Parking_Space, Security, Amenities,
               Facing, Owner_Type FROM house_data WHERE Email = %s
    ''', (user_email,))
    data = cursor.fetchall()

    cursor.close()
    connection.close()

    user_info = get_user_info(user_email)
    return render_template('history.html', data=data, user=user_info)

def get_user_info(email):
    try:
        connection = get_connection()
        cursor = connection.cursor()
        cursor.execute("SELECT Full_Name, Email FROM registers WHERE Email = %s", (email,))
        user = cursor.fetchone()
        return {'name': user[0], 'email': user[1]} if user else None
    finally:
        cursor.close()
        connection.close()

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jun/2025 18:42:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 18:42:11] "GET /static/css/adjustment2.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Jun/2025 18:42:11] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [07/Jun/2025 18:42:11] "GET /static/images/Designer1.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [07/Jun/2025 18:42:20] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [07/Jun/2025 18:42:22] "GET /interaction HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 18:42:22] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [07/Jun/2025 18:42:22] "GET /static/css/adjustment2.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Jun/2025 18:42:22] "GET /static/images/profile.jpeg HTTP/1.1" 304 -


🚨 Error: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''


127.0.0.1 - - [07/Jun/2025 18:42:53] "POST /house_data HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2025 18:42:53] "GET /static/css/adjustment2.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Jun/2025 18:42:53] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [07/Jun/2025 18:42:53] "GET /static/images/profile.jpeg HTTP/1.1" 304 -
